<h1>Exercise 5: Identify the Number Game</h1> 

### The Challenge
Let's play a game! The agent will try to guess a number you are thinking of.

**Goal:** Build a "Guess the Number" agent that intelligently adjusts its guesses based on hints.

**Requirements:**
1. **State**: Track the `target` number, current `lowerBound` & `upperBound`, `guesses` list, and `attempts` count.
2. **Setup Node**: Initialize bounds (e.g., 1 to 20) and attempt counter.
3. **Guess Node**: Pick a random number within the current bounds.
4. **Hint Node**: Compare guess to target.
    - If correct -> "foundIt".
    - If too low -> Update `lowerBound` to the guess + 1 (or just the guess).
    - If too high -> Update `upperBound` to the guess - 1.
    - If too many attempts -> "over".
5. **Loop**: Continue guessing until found or attempts exhausted.

### The Solution
This uses a feedback loop where the agent's specific state (bounds) changes every cycle.

In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
import random

### Step 1: The Game State
Everything the player needs to know.

In [2]:
class AgentState(TypedDict):
    target: int         # The number we want to guess
    playerName: str     # Name of the player
    guesses: List[int]  # History of guesses
    attempts: int       # How many tries so far
    lowerBound: int     # Current minimum possible number
    upperBound: int     # Current maximum possible number
    result: str         # Status ('foundIt', 'lower', 'higher', 'over')

### Step 2: Game Logic Nodes
We split the logic into: **Setup**, **Guessing**, and **Evaluation (Hint)**.

In [3]:
def setupNode(state: AgentState) -> AgentState:
    """Initialize the game board"""
    print(f"--- Starting Game for {state['playerName']} ---")
    state["attempts"] = 0
    state["lowerBound"] = 1
    state["upperBound"] = 20
    state["guesses"] = []
    print(f"Target is: {state['target']}. Range: {state['lowerBound']}-{state['upperBound']}")
    return state

In [4]:
def guessNode(state: AgentState) -> AgentState:
    """Agent makes a guess based on current bounds"""
    # Increment attempt
    state["attempts"] += 1
    
    # Smart guess: pick random within the narrowed range
    guess = random.randint(state["lowerBound"], state["upperBound"])
    
    print(f"Attempt {state['attempts']}: Boss, I guess {guess}.")
    state["guesses"].append(guess)
    return state

In [5]:
def hintNode(state: AgentState) -> AgentState:
    """Evaluates the guess and updates bounds"""
    # Get the latest guess
    last_guess = state["guesses"][-1]
    target = state["target"]
    
    # Check for Max Attempts (Game Over condition)
    if state["attempts"] >= 7:
        state["result"] = "over"
        return state
    
    # Check if correct
    if last_guess == target:
        state["result"] = "foundIt"
        return state
    
    # If guess is too high, the target must be lower
    # So we set the new Upper Bound to guess - 1
    elif last_guess > target:
        state["upperBound"] = last_guess - 1
        state["result"] = "lower"
        print(f"-> Too high! New range: {state['lowerBound']} to {state['upperBound']}")
        return state
    
    # If guess is too low, the target must be higher
    # So we set the new Lower Bound to guess + 1
    else: # last_guess < target
        state["lowerBound"] = last_guess + 1
        state["result"] = "higher"
        print(f"-> Too low! New range: {state['lowerBound']} to {state['upperBound']}")
        return state

### Step 3: The Brain (Router)
Decides to stop or loop based on `state["result"]`.

In [6]:
def decisionLogic(state: AgentState) -> str:
    """Routes the graph based on the result""" 
    result = state["result"]
    
    if result == "over":
        print("\n--- Game Over: Attempts Exhausted ---")
        return "exitEdge"
    
    elif result == "foundIt":
        print(f"\n--- SUCCESS! Found {state['target']} in {state['attempts']} tries. ---")
        return "exitEdge"
    
    else: 
        # If 'lower' or 'higher', we need to guess again
        return "continueEdge"

### Step 4: Build the Graph
Cycle: `Setup -> Guess -> Hint -> (Decision) -> Guess ...`

In [7]:
graph = StateGraph(AgentState)

graph.add_node("setup", setupNode)
graph.add_node("guess", guessNode)
graph.add_node("hint", hintNode)

graph.add_edge(START, "setup")
graph.add_edge("setup", "guess")
graph.add_edge("guess", "hint")

graph.add_conditional_edges(
    "hint",
    decisionLogic,
    {
        "exitEdge": END,
        "continueEdge": "guess" # Loops back to the guessing node
    }
)

app = graph.compile()

### Step 5: Play!
We set the target to 13. Let's see if the agent can find it.

In [8]:
input_data = {
    "playerName": "Student", 
    "target": 13
}

result = app.invoke(input_data)

--- Starting Game for Student ---
Target is: 13. Range: 1-20
Attempt 1: Boss, I guess 8.
-> Too low! New range: 9 to 20
Attempt 2: Boss, I guess 10.
-> Too low! New range: 11 to 20
Attempt 3: Boss, I guess 15.
-> Too high! New range: 11 to 14
Attempt 4: Boss, I guess 13.

--- SUCCESS! Found 13 in 4 tries. ---


### Analysis
Notice how the range shrinks? That's the state updating dynamically. This is a basic form of an "intelligent" agent that learns from the environment (the hints) to improve its next action.